## Maxima

Shims to run Maxima from Python.

Defaults to symbolic computation (exact)

use `float()` to force approx evaluation

In [95]:
import subprocess as sub
import sys
import re
import numpy as np

In [128]:
# http://maxima.sourceforge.net/docs/manual/maxima_singlepage.html#SEC7


def run_maxima(cmd, echo=False) :
    if echo :
        print("\n", cmd, ":")
    
    cmd = ['maxima', '-r'] + [cmd + ';']
    outBytes = sub.run(cmd, stdout=sub.PIPE).stdout
    out = outBytes.decode("utf-8")
    out = re.sub('\n', '', out)
    out = re.sub('.*information\.', '', out)
    lines = re.sub("\(\%", "\n(", out)
    lines = re.sub("[ \t]+", " ", lines)
    lines = re.sub("--\n--", "--\n    ", lines)
    lines = re.sub("--", "--\n    ", lines)
    lines = re.sub("\(i2\)", "", lines)
    lines = re.sub(" --\n    --\n", "", lines)
    lines = re.sub("--\n    -", "--\n    ", lines)
    print(lines)
    #return lines

def eval_maxima(cmd) :
    cmd = "float(" + cmd + ")"
    run_in_maxima(cmd)


def degs_to_rads(degs) :
    return degs * np.pi / 180


def degs_to_rads_str(degs) :
    return str(degs) + "* %pi/180"

In [130]:
cmd = 'print ("I was called as", _)'
run_maxima(cmd)

cmd = "2+3*4/5"
run_maxima(cmd, echo=True)
print("or")
eval_maxima(cmd)


(i1) I was called as _ 
(o1) _
 

 2+3*4/5 :

(i1) 22
(o1) --
     5
 
or

(i1) 
(o1) 4.4
 


In [132]:
cmd = "32**(1/5)"
run_maxima(cmd)

cmd = "sin(%pi/3)"
run_maxima(cmd, echo=True)

# tan 30, but have to pass in rads
rads = degs_to_rads_str(30)
cmd = "tan({})".format(rads)
run_maxima(cmd)

# Returns the argument if it's the simplest form
cmd = "%e^4"
run_maxima(cmd)

# log is ln
# Returns decimal if decimals in
cmd = "log(3.4)"
run_maxima(cmd)



(i1) 
(o1) 2
 

 sin(%pi/3) :

(i1) sqrt(3)
(o1)    --
     2
 

(i1) 1
(o1)    --
     sqrt(3)
 

(i1) 4
(o1) %e
 

(i1) 
(o1) 1.223775431622115
 


In [135]:
cmd = "6.223015277861141 * 10^13"
run_maxima(cmd)

# e != %e
cmd = "6.223015277861141" + "e13"
run_maxima(cmd)


(i1) 
(o1) 6.223015277861141E+13
 

(i1) 
(o1) 6.223015277861141E+13
 


In [142]:
cmd = "sqrt(325)"
run_maxima(cmd)
eval_maxima(cmd)


cmd = "cos(5*%pi/6)"
run_maxima(cmd)

# Multiply last result by 2
cmd = cmd + "; 2*%"
run_maxima(cmd)


(i1) 
(o1) 5 sqrt(13)
 

(i1) 
(o1) 18.02775637731994
 

(i1) sqrt(3)
(o1) -    --
     2
 

(i1) sqrt(3)
(o1) -    --
     2
(o2) - sqrt(3)
(i3) 


In [154]:
# Assignment with :
# = is for equation constraints
cmd = "a : cos(5*%pi/6);" \
        + "a: 2;" \
        + "a: a*a;" \
        + "b: 1;" \
        + "a"
run_maxima(cmd)


cmd = "a:1; c:2; values"
run_maxima(cmd)

cmd = "a:1; c:2; kill(a)"
run_maxima(cmd)


# subst is "pass arg x=3 to c"
cmd = "c: x^2+3;" \
        + "subst(3,x,c)"
run_maxima(cmd)
# == 9+3 = 12


(i1) sqrt(3)
(o1) -    --
     2
(o2) 2
(o3) 4
(o4) 1
(o5) 4
(i6) 

(i1) 
(o1) 1
(o2) 2
(o3) [a, c]
(i4) 

(i1) 
(o1) 1
(o2) 2
(o3) done
(i4) 

(i1) 2
(o1) x + 3
(o2) 12
(i3) 


In [161]:
# Functions
cmd = "f(x) := (x^4+x+1)/(x^3-13*x+12)"
run_maxima(cmd + "; f(2); f(100)")
run_maxima(cmd + "; f(3)") # undefined at x=3

run_maxima("f(x) := 1; functions")


(i1) 4 x + x + 1
(o1) f(x) :=          --
     3 x - 13 x + 12 19
(o2) - --
     6 33333367
(o3)        332904
(i4) 

(i1) 4 x + x + 1
(o1) f(x) :=          --
     3 x - 13 x + 12expt: undefined: 0 to a negative exponent.#0: f(x=3) --
     an error. To debug this try: debugmode(true);
(i3) 

(i1) 
(o1) f(x) := 1
(o2) [f(x)]
(i3) 


In [167]:
# Collections

cmd = "a:makelist(i^2, i, 1, 20)"
run_maxima(cmd)
cmd = "makelist(i, i, 1, 19, 2)"
run_maxima(cmd)
cmd = "makelist(i, i, 2, 19, 3)"
run_maxima(cmd)
cmd = "a:makelist(i, i, 2, 19, 3);" \
        + "makelist(i+1, i, a)"
run_maxima(cmd)


(i1) 
(o1) [1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400]
 

(i1) 
(o1) [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]
 

(i1) 
(o1) [2, 5, 8, 11, 14, 17]
 

(i1) 
(o1) [2, 5, 8, 11, 14, 17]
(o2) [3, 6, 9, 12, 15, 18]
(i3) 


In [173]:
# Vectors

cmd = "v:transpose([2,3,6])"
run_maxima(cmd)

cmd = "v:transpose([2,3,6]);" \
        + "print(); print(); 3*v"
run_maxima(cmd)

cmd = "v:transpose([1,2,3]);" \
        "w:transpose([1,2,3]);" \
        + "v . w"
run_maxima(cmd)

cmd = "P:matrix( [2,-3,3], [-2,1,2], [1,-1,4])"
run_maxima(cmd)


(i1) [ 2 ] [ ]
(o1) [ 3 ] [ ] [ 6 ]
 

(i1) [ 2 ] [ ]
(o1) [ 3 ] [ ] [ 6 ]
(o2) []
(o3) [] [ 6 ] [ ]
(o4) [ 9 ] [ ] [ 18 ]
(i5) 

(i1) [ 1 ] [ ]
(o1) [ 2 ] [ ] [ 3 ] [ 1 ] [ ]
(o2) [ 2 ] [ ] [ 3 ]
(o3) 14
(i4) 

(i1) [ 2 - 3 3 ] [ ]
(o1) [ - 2 1 2 ] [ ] [ 1 - 1 4 ]
 


In [175]:
# SPEED OF LIGHT BABY
cmd = "load(physconst);" \
        + "float(%%c)"
    
run_maxima(cmd)


(i1) 
(o1) /usr/share/maxima/5.41.0/share/physics/physconst.mac 299792458 m
(o2)       --
     s
(i3) 


In [179]:
# SOLVER

cmd = "sol:solve(x^3 -4*x^2 + 2*x+4 = 0)"
run_maxima(cmd)

cmd = "sol:solve(x^3 -4*x^2 + 2*x+4 = 0);" \
        + "rhs(sol[1])"
run_maxima(cmd)

# SIMULT!
cmd = "solve(" \
        + "[y = x^2 - 6*x + 9," \
        + "(x-3)^2 + (y-2)^2 = 3], " \
        + "[x,y])"
run_maxima(cmd)


(i1) 
(o1) [x = 1 - sqrt(3), x = sqrt(3) + 1, x = 2]
 

(i1) 
(o1) [x = 1 - sqrt(3), x = sqrt(3) + 1, x = 2]
(o2) 1 - sqrt(3)
(i3) 

(i1) sqrt(5) - 5 sqrt(5) + 3
(o1) [[x = -       --
    , y =       --
    ], 2 2 sqrt(5) + 5 sqrt(5) - 3 sqrt(5) - 7 sqrt(5) - 3[x =       --
    , y = -       --
    ], [x = -       --
    , y = -       --
    ], 2 2 2 2 sqrt(5) + 7 sqrt(5) + 3[x =       --
    , y =       --
    ]] 2 2
 


In [183]:
# Numerical approx

# in [0,1]:
cmd = "f(x)=x^9 + 2*x - 1;" \
        + "find_root(f(x), x, 0, 1)"
run_maxima(cmd)

# Real and complex
cmd = "allroots(x^9+2*x-1=0)"
run_maxima(cmd)


(i1) 9
(o1) f(x) = x + 2 x - 1
(o2) find_root(f(x), x, 0.0, 1.0)
(i3) 

(i1) 
(o1) [x = 0.4990401803569676, x = 0.4207825521114528 %i - 1.058521241857364, x = (- 0.4207825521114528 %i) - 1.058521241857364, x = 1.01795501023027 %i - 0.4714669586482951, x = (- 1.01795501023027 %i) - 0.4714669586482951, x = 1.024451854329253 %i + 0.3548245783739534, x = 0.3548245783739534 - 1.024451854329253 %i, x = 0.4326584474822993 %i + 0.9256435319532224, x = 0.9256435319532224 - 0.4326584474822993 %i]
 


In [184]:
# CALCULUS

cmd = "diff(cos(x)/(x^2+3),x)"
run_maxima(cmd )


(i1) sin(x) 2 x cos(x)
(o1) (-    --
    ) -       --
     2 2 2 x + 3 (x + 3)
 
